# AUC Explorations
In this notebook we try to establish what the difference is when you calculate an AUC score for several models, between
  1. calculating the AUC on the combined model outputs ("Combined AUC"), or
  1. calculating the AUC on each model and take the average ("Average AUC")
 
There is a lot of discussion on the forums about the relative validity of these two approaches.
  
We investigate this question by simulating submission data and calculating the AUC by the two methods above. The simulation is based on a model that generates prediction probabilities for three patients. The model captures these features:
  * the class probabilities (what fraction of samples are of class 1)
  * prediction accuracy
 
To model the prediction accuracy, we need to pick a distribution over [0, 1], since the predictions are probabilities. We choose the beta distribution mostly because it looks about right. (See the plots below to convince yourself of this.)

$$ \mathrm{pdf}_{\mathrm{beta}}(x; a, b) = \frac{\Gamma(a+b) \, x^{(a-1)} \, (1-x)^{(b-1)}}{\Gamma(a) \, \Gamma(b)} $$

The general idea is that when presented with a class 0 sample, the prediction is a draw from a beta distribution that is skewed toward 0. For class 1, the skew is toward 1. By modifying the parameters of the beta distribution we can adjust its skew and kurtosis in a way that plausibly captures the prediction accuracy of a solution.

We use the same parameters $a, b$ in both cases, but swap them, using the property of the beta distribution that

$$ \mathrm{pdf}_{\mathrm{beta}}(1-x;a,b) = \mathrm{pdf}_{\mathrm{beta}}(x;b,a) $$

In other words, we make the simplifying assumption that the accuracies for class 0 and class 1 are the same.

The code below is set up for three experiments:

  1. different class probabilities, same accuracies
  1. same class probabilities, different accuracies
  1. different class probabilities, different accuracies

In [ ]:
%matplotlib inline

import warnings

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import beta
import seaborn as sns
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [ ]:
def quantile_normalize(trg, ref):
    """Quantile normalize target ndarray to match distribution of reference ndarray"""
    
    # We sort the target in a way that lets us unsort it later
    idx = np.argsort(trg)
    
    # Calculate the percentile points of the target
    n_trg = len(trg)
    percs = np.arange(n_trg) * 100 / (n_trg - 1)
    
    # Calculate the probabilities at the same percentile points in the reference
    probs = np.percentile(ref, percs)
    
    # Now unsort those probabilities
    idx2 = np.argsort(idx)
    trg2 = probs[idx2]
    
    return trg2

## Set the model parameters
Change the `experiment` variable below to use one of the preset experiments, or tweak the parameters yourself.

Smaller values of `b` correspond to less accurate models. The value of `a` is probably best left at 2.

In [ ]:
n_patients = 3
experiment = 3
if experiment == 1:  # different class probs, same accuracy
    class_probs = [.1, .2, .3]
    a = [2, 2, 2]
    b = [3, 3, 3]
elif experiment == 2:  # same class probs, different accuracies
    class_probs = [.15, .15, .15]
    a = [2, 2, 2]
    b = [2, 4, 6]
elif experiment == 3:  # different class probs, different accuracies
    class_probs = [.15, .20, .25]
    a = [2, 2, 2]
    b = [3.5, 4.0, 4.5]

The plots below are just a sanity check to show the distribution for class 0 predictions generated by our model.

In [ ]:
# Sanity check: generate draws and plot
fig = plt.figure(figsize=(9, 3))
ax = {}
for p in range(n_patients):
    if p == 0:
        ax[p] = plt.subplot(1, 3, p + 1)
    else:
        ax[p] = plt.subplot(1, 3, p + 1, sharey=ax[0])
    plt.xlim(0, 1)
    plt.title('Patient {} - Class 0'.format(p + 1))
    probs = beta.rvs(a[p], b[p], size=1000)
    sns.distplot(probs, kde=None, fit=beta)

## Generate sample points
We generate predictions for class 0 and class 1 points in proportion to the class probabilites of the model. As described above, the predictions are generated by drawing from the beta distribution. 

In [ ]:
n_points = [1000, 2000, 4000]
y_score = []
y_true = []
for p in range(n_patients):
    n_1 = int(n_points[p] * class_probs[p] + 0.5)
    n_0 = n_points[p] - n_1
    
    # Class 0 predictions
    y_s = list(beta.rvs(a[p], b[p], size=n_0))
    y_t = [0] * n_0
    
    # Class 1 predictions
    y_s += list(beta.rvs(b[p], a[p], size=n_1))
    y_t += [1] * n_1
    
    y_score.append(y_s)
    y_true.append(y_t)

And again, here are some plots to sanity check that the distribution of probabilities (for both classes now) look plausible. If you're not convinced, plot some of your submission accuracies and compare.

In [ ]:
fig = plt.figure(figsize=(9, 3))
ax = {}
for p in range(n_patients):
    plt.subplot(1, 3, p + 1)
    if p == 0:
        ax[p] = plt.subplot(1, 3, p + 1)
    else:
        ax[p] = plt.subplot(1, 3, p + 1, sharey=ax[0])
    plt.xlim(0, 1)
    plt.title('Patient {}'.format(p + 1))
    sns.distplot(y_score[p])

## Calculate AUC

### Method 1: Average of individual AUC scores

In [ ]:
auc = {}
for p in range(n_patients):
    auc[p] = roc_auc_score(y_true[p], y_score[p])
    print('Patient {} AUC: {:.3f}'.format(p + 1, auc[p]))
auc_mean = sum(auc.values()) / len(auc)
print('--------------------')
print('Average AUC:   {:.3f}'.format(auc_mean))

### Method 2: Combined AUC score

In [ ]:
y_score_all = np.concatenate(y_score)
y_true_all = np.concatenate(y_true)
auc_all = roc_auc_score(y_true_all, y_score_all)
print('Combined AUC:  {:.3f}'.format(auc_all))

### Method 3: Normalize before calculating combined AUC score
We use quantile normalization to give the same distribution to each of the individual patient's predictions.

In [ ]:
y_score_norm = []
for p in range(n_patients):
    y_score_norm.append(quantile_normalize(y_score[p], y_score[2]))
y_score_all_norm = np.concatenate(y_score_norm)
auc_all_norm = roc_auc_score(y_true_all, y_score_all_norm)
print('Normed AUC:    {:.3f}'.format(auc_all_norm))

## Discussion

For most combinations of class probabilities and model accuracies, `Combined AUC > Normed AUC > Average AUC`. 

## Conclusion

Even if you believe that the combined AUC is a flawed scoring system, you probably can't fix it by normalizing the distributions of the individual patient's predictions.